In [1]:
!pip install torch transformers rdkit torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertConfig, BertModel, BertLMHeadModel
from torch.optim import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from rdkit import Chem
import copy
import math
from torch.utils.data import DataLoader, Dataset

2025-12-12 11:50:36.521406: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765540236.672253      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765540236.723666      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [4]:
from torch_geometric.data import InMemoryDataset
class PubChemDataset(InMemoryDataset):
    def __init__(self, path):
        super(PubChemDataset, self).__init__()
        self.data, self.slices = torch.load(path, weights_only=False)
    
    def __getitem__(self, idx):
        return self.get(idx)

In [5]:
raw_dataset = PubChemDataset('/kaggle/input/pubchem324k-dataset/train.pt')

In [6]:
ldmol_dataset = []
for data in raw_dataset:
    ldmol_dataset.append({
            'smiles': data.smiles,
            'text': data.text
        })

In [7]:
print(ldmol_dataset[0:2])

[{'smiles': 'C1=CC=C(C=C1)C(CN)O', 'text': 'The molecule is the simplest member of the class of phenylethanolamines that is 2-aminoethanol bearing a phenyl substituent at the 1-position. The parent of the phenylethanolamine class. It has a role as a human metabolite. It is a conjugate base of a phenylethanolaminium.\nThe molecule is a natural product found in Peucedanum palustre, Pseudo-nitzschia multistriata, and other organisms with data available.\nThe molecule is also used in chemical industry.'}, {'smiles': 'CC(=C)C(CC1=C2C(=C(C3=C1OC4=C(C3=O)C=C(C=C4)O)O)C=CC(O2)(C)C)OO', 'text': 'The molecule is a member of the class of pyranoxanthones that is 2H,6H-pyrano[3,2-b]xanthen-6-one substituted by a 2-hydroperoxy-3-methylbut-3-en-1-yl moiety at position 12, hydroxy groups at positions 5 and 8 and geminal methyl groups at position 2. It is isolated from the stem bark of Calophyllum brasiliense and exhibits significant inhibitory activity against 12-O-tetradecanoylphorbol-13-acetate indu

In [8]:
class SMILESTokenizer:
    def __init__(self, vocab_list):
        self.vocab = vocab_list
        self.max_len = 512

        self.pad_token = '[PAD]'
        self.unk_token = '[UNK]'
        self.cls_token = '[CLS]'
        self.sep_token = '[SEP]'
        
        self.token_to_id = {token: i for i, token in enumerate(vocab_list)}
        self.id_to_token = {i: token for token, i in self.token_to_id.items()}
        
        self.pad_token_id = self.token_to_id.get(self.pad_token, 0)
        self.unk_token_id = self.token_to_id.get(self.unk_token, 1)
        self.cls_token_id = self.token_to_id.get(self.cls_token, 2)
        self.sep_token_id = self.token_to_id.get(self.sep_token, 3)

    def tokenize(self, smiles):
        tokens = []
        i = 0
        while i < len(smiles):
            match = None
            for j in range(len(smiles), i, -1):
                sub = smiles[i:j]
                if len(sub) == 1 and sub in self.token_to_id:
                    match = sub
                    break
                if sub in self.token_to_id and sub.startswith('##'):
                    match = sub
                    break
                if sub in self.token_to_id:
                    match = sub
                    break

            if match:
                tokens.append(match)
                i += len(match)
            else:
                if smiles[i] in self.token_to_id:
                    tokens.append(smiles[i])
                else:
                    tokens.append(self.unk_token)
                i += 1
        return tokens

    def __call__(self, smiles, padding='max_length', truncation=True, max_length=512, return_tensors='pt'):
        """
        Main encoding function.
        """
        tokens = [self.cls_token] + self.tokenize(smiles) + [self.sep_token]
        
        if truncation and len(tokens) > max_length:
            tokens = tokens[:max_length - 1] + [self.sep_token] # Ensure SEP is at the end
        input_ids = [self.token_to_id.get(t, self.unk_token_id) for t in tokens]
        attention_mask = [1] * len(input_ids)
        if padding == 'max_length':
            padding_len = max_length - len(input_ids)
            input_ids += [self.pad_token_id] * padding_len
            attention_mask += [0] * padding_len
        input_ids = torch.tensor(input_ids).unsqueeze(0)
        attention_mask = torch.tensor(attention_mask).unsqueeze(0)
        return {'input_ids': input_ids, 'attention_mask': attention_mask}

In [9]:
class LDMolAutoencoder(nn.Module):
    def __init__(self, encoder_config_dict, decoder_config_dict, pretrain_config, tokenizer):
        super().__init__()
        self.pretrain_config = pretrain_config
        self.device_param = nn.Parameter(torch.empty(0))
        self.tokenizer = tokenizer
        decoder_config_dict['add_cross_attention'] = True
        decoder_config_dict['is_decoder'] = True
        
        self.encoder_config = BertConfig(**encoder_config_dict)
        self.decoder_config = BertConfig(**decoder_config_dict)
        self.encoder = BertModel(self.encoder_config)
        self.decoder = BertLMHeadModel(self.decoder_config)
        self.compression = nn.Linear(
            self.encoder_config.hidden_size, 
            self.pretrain_config['embed_dim']
        )
        
        # Projection Head for Contrastive Loss
        self.projection_head = nn.Linear(
            self.encoder_config.hidden_size,
            self.pretrain_config['embed_dim']
        )

        # 3. Momentum Components
        self.encoder_momentum = copy.deepcopy(self.encoder)
        self.projection_head_momentum = copy.deepcopy(self.projection_head)
        
        # Freeze momentum parameters
        for param in self.encoder_momentum.parameters():
            param.requires_grad = False
        for param in self.projection_head_momentum.parameters():
            param.requires_grad = False
            
        # 4. Negative Sample Queue
        self.queue_size = self.pretrain_config['queue_size']
        self.embed_dim = self.pretrain_config['embed_dim']
        self.register_buffer("queue", torch.randn(self.embed_dim, self.queue_size))
        self.queue = F.normalize(self.queue, dim=0)
        self.register_buffer("queue_ptr", torch.zeros(1, dtype=torch.long))

    @property
    def device(self):
        return self.device_param.device
    def _momentum_update(self):
        m = self.pretrain_config['momentum']
        with torch.no_grad():
            for param_q, param_k in zip(self.encoder.parameters(), self.encoder_momentum.parameters()):
                param_k.data = param_k.data * m + param_q.data * (1. - m)
            for param_q, param_k in zip(self.projection_head.parameters(), self.projection_head_momentum.parameters()):
                param_k.data = param_k.data * m + param_q.data * (1. - m)

    def _dequeue_and_enqueue(self, keys):
        batch_size = keys.shape[0]
        ptr = int(self.queue_ptr)
        if ptr + batch_size <= self.queue_size:
            self.queue[:, ptr:ptr + batch_size] = keys.T
            ptr = (ptr + batch_size) % self.queue_size
        else:
            tail = self.queue_size - ptr
            self.queue[:, ptr:] = keys[:tail].T
            self.queue[:, :batch_size-tail] = keys[tail:].T
            ptr = batch_size - tail
        self.queue_ptr[0] = ptr
    def forward(self, input_ids, attention_mask, labels=None, input_ids_aug=None, attention_mask_aug=None):
        # Contrastive Learning (InfoNCE)
        # Query Encoder (Canonical SMILES)
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :] 
        q = self.projection_head(cls_output)
        q = F.normalize(q, dim=1)

        # Key Encoder (Momentum) (Enumerated SMILES)
        with torch.no_grad():
            self._momentum_update()
            outputs_aug = self.encoder_momentum(input_ids=input_ids_aug, attention_mask=attention_mask_aug)
            cls_output_aug = outputs_aug.last_hidden_state[:, 0, :]
            k = self.projection_head_momentum(cls_output_aug)
            k = F.normalize(k, dim=1)

        # Calculate Contrastive Logits
        l_pos = torch.einsum('nc,nc->n', [q, k]).unsqueeze(-1)
        l_neg = torch.einsum('nc,ck->nk', [q, self.queue.clone().detach()])

        logits = torch.cat([l_pos, l_neg], dim=1)
        logits /= self.pretrain_config['temp']
        contrastive_targets = torch.zeros(logits.shape[0], dtype=torch.long, device=self.device)
        loss_con = F.cross_entropy(logits, contrastive_targets)
        
        self._dequeue_and_enqueue(k)

        # Reconstruction (Autoencoder)
        latent_z = self.compression(outputs.last_hidden_state) 
        decoder_outputs = self.decoder(
            input_ids=input_ids,          
            attention_mask=attention_mask,
            encoder_hidden_states=latent_z, 
            encoder_attention_mask=attention_mask,
            labels=labels                 
        )
        
        loss_dec = decoder_outputs.loss
        total_loss = loss_con + loss_dec 
        return {
            'loss': total_loss,
            'loss_con': loss_con,
            'loss_dec': loss_dec
        }

In [10]:
vocab_path = '/kaggle/input/vocab-bpe-300/vocab_bpe_300_sc.txt'
vocab_list = []
with open(vocab_path, "r", encoding="utf-8") as f:
    vocab_list = [line.strip() for line in f.readlines()]

print(vocab_list)

['[PAD]', '[UNK]', '[CLS]', '[SEP]', '##.', '##/', '##\\', '##@', '##@@', '###', '##%', '##(', '##)', '##+', '##-', '##=', '##[', '##]', '##0', '##1', '##2', '##3', '##4', '##5', '##6', '##7', '##8', '##9', '##A', '##B', '##C', '##D', '##E', '##F', '##G', '##H', '##I', '##K', '##L', '##M', '##N', '##O', '##P', '##R', '##S', '##T', '##U', '##V', '##W', '##X', '##Y', '##Z', '##a', '##b', '##c', '##d', '##e', '##f', '##g', '##h', '##i', '##l', '##m', '##n', '##o', '##p', '##r', '##s', '##t', '##u', '##y', '##cc', '##CC', '##(C', '##c1', '##O)', '##=O)', '##(=O)', '##ccc', '##(C)', '##c2', '##C(=O)', '##)cc', '##+]', '##[N', '##CCC', '##c1cc', '##[NH', '##c(', '##c1ccc', '##C(', '##c3', '##2)', '##F)', '##C1', '##CCCC', '##c2cc', '##OC', '##c1cccc', '##NC(=O)', '##)cc1', '##CC1', '##(=O)N', '##(C)C', '##-]', '##CO', '##c1ccc(', '##[O', '##[O-]', '##n1', '##[NH+]', '##c2ccc', '##3)', '##(Cl', '##(F)', '##c1ccccc1', '##ccccc', '##CCO', '##C(=O)N', '##2+]', '##[NH2+]', '##c2ccccc', '##(CC', '

In [11]:
class LDMolDataset(Dataset):
    def __init__(self, data_list, tokenizer, max_len=512):
        self.data = data_list
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.data)
        
    def enumerate_smiles(self, smiles):
        """Generates a random SMILES traversal (SMILES Enumeration)."""
        mol = Chem.MolFromSmiles(smiles)
        if mol is None: return smiles
        try:
            return Chem.MolToSmiles(mol, doRandom=True)
        except:
             return smiles

    def __getitem__(self, idx):
        item = self.data[idx]
        smiles_can = item['smiles']
        smiles_aug = self.enumerate_smiles(smiles_can)
        enc_can = self.tokenizer(smiles_can, max_length=self.max_len)
        enc_aug = self.tokenizer(smiles_aug, max_length=self.max_len)
        labels = enc_can['input_ids'].squeeze(0).clone()
        labels[labels == self.tokenizer.pad_token_id] = -100
        
        return {
            'input_ids': enc_can['input_ids'].squeeze(0),
            'attention_mask': enc_can['attention_mask'].squeeze(0),
            'input_ids_aug': enc_aug['input_ids'].squeeze(0),
            'attention_mask_aug': enc_aug['attention_mask'].squeeze(0),
            'labels': labels
        }

In [12]:
import gc
def save_checkpoint(model, optimizer, epoch, loss, path):
    """Saves model and optimizer states to a checkpoint file."""
    print(f"\nSaving checkpoint for epoch {epoch+1} to {path}...")
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, path)
    print("Checkpoint saved.")
def train_ldmol_model(encoder_cfg, decoder_cfg, train_cfg, data_list, tokenizer):
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()
    model = LDMolAutoencoder(encoder_cfg, decoder_cfg, train_cfg, tokenizer)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.train()

    dataset = LDMolDataset(data_list, tokenizer)
    dataloader = DataLoader(dataset, batch_size=train_cfg['batch_size'], shuffle=True)

    optimizer = AdamW(
        model.parameters(), 
        lr=train_cfg['optimizer']['lr'], 
        weight_decay=train_cfg['optimizer']['weight_decay']
    )

    num_epochs = train_cfg['schedular']['epochs']
    total_steps = len(dataloader) * num_epochs
    warmup_steps = int(len(dataloader) * train_cfg['schedular']['warmup_epochs']) # Use steps from epochs

    scheduler = get_cosine_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=warmup_steps, 
        num_training_steps=total_steps
    )

    print(f"Starting LDMol Autoencoder (Stage 1) training on {device}...")
    print(f"Total steps: {total_steps}, Warmup steps: {warmup_steps}")
    
    for epoch in range(num_epochs):
        total_loss = 0
        input_ids, attention_mask, input_ids_aug, attention_mask_aug, labels = None, None, None, None, None
        for step, batch in enumerate(dataloader):
            # Move batch to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            input_ids_aug = batch['input_ids_aug'].to(device)
            attention_mask_aug = batch['attention_mask_aug'].to(device)
            labels = batch['labels'].to(device)

            # Forward Pass
            optimizer.zero_grad()
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels,
                input_ids_aug=input_ids_aug,
                attention_mask_aug=attention_mask_aug
            )

            loss = outputs['loss']
            
            # Backward Pass and Update
            loss.backward()
            optimizer.step()
            scheduler.step()

            total_loss += loss.item()

            if step % 50 == 0:
                print(f"Epoch {epoch+1}/{num_epochs} | Step {step} | LR: {optimizer.param_groups[0]['lr']:.2e} | Loss: {loss.item():.4f} "
                      f"(Con: {outputs['loss_con'].item():.4f}, Dec: {outputs['loss_dec'].item():.4f})")
        avg_loss = total_loss / len(dataloader)
        save_checkpoint(model, optimizer, epoch, avg_loss, path=f'ldmol_stage1_epoch_{epoch+1}.pth')
        
        # Clear unused memory after each epoch
        if device.type == 'cuda':
            del input_ids, attention_mask, input_ids_aug, attention_mask_aug, labels
            torch.cuda.empty_cache()
            gc.collect()
    print("Training Complete.")
    return model

In [13]:
if __name__ == "__main__":
    enc_config = {
      "architectures": ["BertForMaskedLM"], "attention_probs_dropout_prob": 0.1,
      "hidden_act": "gelu", "hidden_dropout_prob": 0.1, 
      "hidden_size": 768, 
      "initializer_range": 0.02, "intermediate_size": 3072, "layer_norm_eps": 1e-12,
      "max_position_embeddings": 512, "model_type": "bert", "num_attention_heads": 12,
      "num_hidden_layers": 6, 
      "pad_token_id": 0, "type_vocab_size": 2, 
      "vocab_size": 300, 
      "fusion_layer": 6, "autoregressive": 0 
    }
    
    dec_config = {
      "architectures": ["BertForMaskedLM"], "attention_probs_dropout_prob": 0.1,
      "hidden_act": "gelu", "hidden_dropout_prob": 0.1, 
      "hidden_size": 512, 
      "initializer_range": 0.02, "intermediate_size": 2048, "layer_norm_eps": 1e-12,
      "max_position_embeddings": 512, "model_type": "bert", "num_attention_heads": 8, 
      "num_hidden_layers": 6, 
      "pad_token_id": 0, "type_vocab_size": 2, 
      "vocab_size": 300, 
      "fusion_layer": 0, 
      "encoder_width": 256, # CORRECTED: Matches pt_config['embed_dim']
      "autoregressive": 0,
      "add_cross_attention": "True"
    }
    pt_config = {
        'embed_dim': 512, 
        'batch_size': 32, 
        'temp': 0.07, 'queue_size': 16384,
        'momentum': 0.995, 'alpha': 0.4, 'bert_config_encoder': './config_encoder.json',
        'schedular': {'sched': 'cosine', 'lr': 1e-4, 'epochs': 5, 'min_lr': 1e-5,
                      'decay_rate': 1, 'warmup_lr': 5e-5, 'warmup_epochs': 20},
        'optimizer': {'opt': 'adamW', 'lr': 1e-4, 'weight_decay': 0.02}
    }
    tokenizer = SMILESTokenizer(vocab_list)
    # Run Training
    trained_model = train_ldmol_model(enc_config, dec_config, pt_config, ldmol_dataset, tokenizer)

Starting LDMol Autoencoder (Stage 1) training on cuda...
Total steps: 1875, Warmup steps: 7500


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch 1/5 | Step 0 | LR: 1.33e-08 | Loss: 6.2187 (Con: 0.1236, Dec: 6.0951)


[11:52:20] WARNING: not removing hydrogen atom without neighbors
[11:52:20] WARNING: not removing hydrogen atom without neighbors
[11:52:57] WARNING: not removing hydrogen atom without neighbors
[11:52:57] WARNING: not removing hydrogen atom without neighbors
[11:53:00] WARNING: not removing hydrogen atom without neighbors
[11:53:00] WARNING: not removing hydrogen atom without neighbors
[11:53:00] WARNING: not removing hydrogen atom without neighbors
[11:53:00] WARNING: not removing hydrogen atom without neighbors
[11:53:18] WARNING: not removing hydrogen atom without neighbors


Epoch 1/5 | Step 50 | LR: 6.80e-07 | Loss: 12.0815 (Con: 7.4002, Dec: 4.6813)


...

Epoch 5/5 | Step 300 | LR: 2.40e-05 | Loss: 4.9754 (Con: 4.9175, Dec: 0.0578)


[13:01:43] WARNING: not removing hydrogen atom without neighbors
[13:02:01] WARNING: not removing hydrogen atom without neighbors
[13:02:01] WARNING: not removing hydrogen atom without neighbors
[13:02:15] WARNING: not removing hydrogen atom without neighbors
[13:02:15] WARNING: not removing hydrogen atom without neighbors
[13:02:29] WARNING: not removing hydrogen atom without neighbors
[13:02:29] WARNING: not removing hydrogen atom without neighbors
[13:02:33] WARNING: not removing hydrogen atom without neighbors
[13:02:33] WARNING: not removing hydrogen atom without neighbors
[13:02:36] WARNING: not removing hydrogen atom without neighbors
[13:02:45] WARNING: not removing hydrogen atom without neighbors
[13:02:49] WARNING: not removing hydrogen atom without neighbors
[13:03:03] WARNING: not removing hydrogen atom without neighbors


Epoch 5/5 | Step 350 | LR: 2.47e-05 | Loss: 4.7446 (Con: 4.7009, Dec: 0.0437)


[13:03:53] WARNING: not removing hydrogen atom without neighbors
[13:04:11] WARNING: not removing hydrogen atom without neighbors



Saving checkpoint for epoch 5 to ldmol_stage1_epoch_5.pth...
Checkpoint saved.
Training Complete.


In [14]:
from transformers import T5EncoderModel, T5Tokenizer
import numpy as np

In [15]:
class MolT5TextEncoder(nn.Module):
    def __init__(self, model_name='laituan245/molt5-large', device='cuda'):
        super().__init__()
        print(f"Loading Text Encoder: {model_name}...")
        self.tokenizer = T5Tokenizer.from_pretrained(model_name, legacy=False)
        self.model = T5EncoderModel.from_pretrained(model_name).to(device)
        self.model.eval()
        self.device = device

    @torch.no_grad()
    def get_text_embeddings(self, text_list, max_len=128):
        inputs = self.tokenizer(
            text_list, 
            padding='max_length', 
            truncation=True, 
            max_length=max_len, 
            return_tensors="pt"
        ).to(self.device)
        
        outputs = self.model(**inputs)
        # Returns [Batch, Seq_Len, Hidden_Dim]
        return outputs.last_hidden_state, inputs.attention_mask

In [16]:
def modulate(x, shift, scale):
    return x * (1 + scale.unsqueeze(1)) + shift.unsqueeze(1)

In [17]:
class TimestepEmbedder(nn.Module):
    """
    Embeds scalar timesteps into vector representations.
    """
    def __init__(self, hidden_size, frequency_embedding_size=256):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(frequency_embedding_size, hidden_size, bias=True),
            nn.SiLU(),
            nn.Linear(hidden_size, hidden_size, bias=True),
        )
        self.frequency_embedding_size = frequency_embedding_size

    @staticmethod
    def timestep_embedding(t, dim, max_period=10000):
        """
        Create sinusoidal timestep embeddings.
        :param t: a 1-D Tensor of N indices, one per batch element.
        :param dim: the dimension of the output.
        :return: an (N, dim) Tensor of positional embeddings.
        """
        half = dim // 2
        freqs = torch.exp(
            -math.log(max_period) * torch.arange(start=0, end=half, dtype=torch.float32) / half
        ).to(device=t.device)
        args = t[:, None].float() * freqs[None]
        embedding = torch.cat([torch.cos(args), torch.sin(args)], dim=-1)
        if dim % 2:
            embedding = torch.cat([embedding, torch.zeros_like(embedding[:, :1])], dim=-1)
        return embedding

    def forward(self, t):
        t_freq = self.timestep_embedding(t, self.frequency_embedding_size)
        t_emb = self.mlp(t_freq)
        return t_emb

In [18]:
class DiTBlock(nn.Module):
    """
    A DiT block with adaptive layer norm (adaLN) and Cross-Attention for text conditioning.
    Based on Figure 6(b) in the LDMol paper.
    """
    def __init__(self, hidden_size, num_heads, text_embed_dim, mlp_ratio=4.0):
        super().__init__()
        self.norm1 = nn.LayerNorm(hidden_size, elementwise_affine=False, eps=1e-6)
        self.attn = nn.MultiheadAttention(hidden_size, num_heads=num_heads, batch_first=True)
        
        # Cross-Attention Layer (New for LDMol)
        self.norm_cross = nn.LayerNorm(hidden_size, elementwise_affine=False, eps=1e-6)
        self.cross_attn = nn.MultiheadAttention(hidden_size, num_heads=num_heads, batch_first=True, kdim=text_embed_dim, vdim=text_embed_dim)
        
        self.norm2 = nn.LayerNorm(hidden_size, elementwise_affine=False, eps=1e-6)
        mlp_hidden_dim = int(hidden_size * mlp_ratio)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_size, mlp_hidden_dim),
            nn.GELU(),
            nn.Linear(mlp_hidden_dim, hidden_size),
        )
        self.adaLN_modulation = nn.Sequential(
            nn.SiLU(),
            nn.Linear(hidden_size, 9 * hidden_size, bias=True) 
        )

    def forward(self, x, c, text_emb, text_mask=None):
        # c is timestep embedding
        shift_msa, scale_msa, gate_msa, shift_cross, scale_cross, gate_cross, shift_mlp, scale_mlp, gate_mlp = self.adaLN_modulation(c).chunk(9, dim=1)
        
        # 1. Self-Attention
        x_skip = x
        x = modulate(self.norm1(x), shift_msa, scale_msa)
        x, _ = self.attn(x, x, x)
        x = x_skip + gate_msa.unsqueeze(1) * x
        
        # 2. Cross-Attention (Conditioning on Text)
        if text_emb is not None:
            x_skip = x
            x = modulate(self.norm_cross(x), shift_cross, scale_cross)
            key_padding_mask = (1.0 - text_mask).bool() if text_mask is not None else None
            
            x, _ = self.cross_attn(query=x, key=text_emb, value=text_emb, key_padding_mask=key_padding_mask)
            x = x_skip + gate_cross.unsqueeze(1) * x
        
        # 3. Feed Forward
        x_skip = x
        x = modulate(self.norm2(x), shift_mlp, scale_mlp)
        x = self.mlp(x)
        x = x_skip + gate_mlp.unsqueeze(1) * x
        
        return x

In [19]:
class LDMolDiT(nn.Module):
    """
    Main Diffusion Transformer Model.
    Input: Latent Z (Batch, L, d_z)
    Condition: Text Embedding (Batch, L_text, d_text) + Timestep
    Output: Noise Prediction (Batch, L, d_z)
    """
    def __init__(self, input_size=512, hidden_size=768, depth=12, num_heads=12, text_embed_dim=1024, max_len=512):
        super().__init__()
        self.input_size = input_size
        self.num_heads = num_heads
        
        # Input Projection (Latent -> Hidden)
        self.x_embedder = nn.Linear(input_size, hidden_size)
        
        # Timestep Embedder
        self.t_embedder = TimestepEmbedder(hidden_size)
        
        # Position Embedding (Learnable)
        self.pos_embed = nn.Parameter(torch.zeros(1, max_len, hidden_size))
        
        # DiT Blocks
        self.blocks = nn.ModuleList([
            DiTBlock(hidden_size, num_heads, text_embed_dim) for _ in range(depth)
        ])
        
        # Final Layer
        self.final_layer = nn.Sequential(
            nn.LayerNorm(hidden_size, elementwise_affine=False, eps=1e-6),
            nn.Linear(hidden_size, hidden_size), # Final AdaLN logic usually involves one last shift/scale
            nn.Linear(hidden_size, input_size)   # Project back to Latent dim
        )
        self.final_adaLN = nn.Sequential(
            nn.SiLU(),
            nn.Linear(hidden_size, 2 * hidden_size, bias=True)
        )
        
        self.initialize_weights()

    def initialize_weights(self):
        # Initialize standard Transformer weights
        def _basic_init(module):
            if isinstance(module, nn.Linear):
                torch.nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0)
        self.apply(_basic_init)
        for block in self.blocks:
            nn.init.constant_(block.adaLN_modulation[-1].weight, 0)
            nn.init.constant_(block.adaLN_modulation[-1].bias, 0)
        nn.init.constant_(self.final_adaLN[-1].weight, 0)
        nn.init.constant_(self.final_adaLN[-1].bias, 0)

    def forward(self, x, t, text_emb, text_mask):
        """
        x: (N, L, d_z) - Noisy Latent
        t: (N,) - Timesteps
        text_emb: (N, L_text, d_text)
        text_mask: (N, L_text)
        """
        # 1. Embed Inputs
        x = self.x_embedder(x) + self.pos_embed[:, :x.shape[1], :]
        t_emb = self.t_embedder(t)
        
        # 2. Transformer Blocks
        for block in self.blocks:
            x = block(x, t_emb, text_emb, text_mask)
            
        # 3. Final Layer
        shift, scale = self.final_adaLN(t_emb).chunk(2, dim=1)
        x = modulate(self.final_layer[0](x), shift, scale)
        x = self.final_layer[2](x) # Output projection
        
        return x

In [20]:
class Stage2Dataset(Dataset):
    def __init__(self, data_list, smiles_tokenizer, max_len=512):
        self.data = data_list
        self.smiles_tokenizer = smiles_tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        smiles = item['smiles']
        text = item['text']
        enc_smiles = self.smiles_tokenizer(smiles, max_length=self.max_len)
        return {
            'input_ids': enc_smiles['input_ids'].squeeze(0),
            'attention_mask': enc_smiles['attention_mask'].squeeze(0),
            'text': text
        }

In [21]:
def train_diffusion_stage(stage1_model, train_cfg, data_list, smiles_tokenizer):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #Autoencoder (Frozen)
    stage1_model.to(device)
    stage1_model.eval()
    for param in stage1_model.parameters():
        param.requires_grad = False
        
    #Text Encoder (Frozen) - MolT5-large
    text_encoder_name = 'laituan245/molt5-base' 
    text_encoder = MolT5TextEncoder(text_encoder_name, device=device)
    
    #Diffusion Transformer (DiT)
    dit_model = LDMolDiT(
        input_size=train_cfg['embed_dim'], 
        hidden_size=768, 
        depth=12, 
        num_heads=12, 
        text_embed_dim=text_encoder.model.config.d_model, 
        max_len=512
    ).to(device)
    dit_model.train()
    optimizer = torch.optim.AdamW(dit_model.parameters(), lr=1e-4, weight_decay=0.0)
    dataset = Stage2Dataset(data_list, smiles_tokenizer)
    dataloader = DataLoader(dataset, batch_size=train_cfg['batch_size'], shuffle=True)
    
    num_epochs = train_cfg['schedular']['epochs']
    print(f"Starting Diffusion (Stage 2) Training on {device}...")
    num_timesteps = 1000
    beta_start = 0.0001
    beta_end = 0.02
    betas = torch.linspace(beta_start, beta_end, num_timesteps).to(device)
    alphas = 1.0 - betas
    alphas_cumprod = torch.cumprod(alphas, dim=0)
    
    for epoch in range(num_epochs):
        total_loss = 0
        for step, batch in enumerate(dataloader):
            smiles_ids = batch['input_ids'].to(device)
            smiles_mask = batch['attention_mask'].to(device)
            text_list = batch['text']
            # A. Get Latent Z from Stage 1 (Frozen)
            with torch.no_grad():
                enc_out = stage1_model.encoder(input_ids=smiles_ids, attention_mask=smiles_mask)
                latent_z = stage1_model.compression(enc_out.last_hidden_state) # Shape: (B, L, 512)
                if np.random.random() < 0.03:
                    text_list = [""] * len(text_list) # Null text
                
                text_emb, text_mask = text_encoder.get_text_embeddings(text_list)
            # B. Add Noise (Forward Diffusion)
            batch_size = latent_z.shape[0]
            t = torch.randint(0, num_timesteps, (batch_size,), device=device).long()
            
            noise = torch.randn_like(latent_z)
            alpha_cum_t = alphas_cumprod[t].view(-1, 1, 1) 
            
            z_noisy = torch.sqrt(alpha_cum_t) * latent_z + torch.sqrt(1 - alpha_cum_t) * noise
            #Predict Noise (DiT Forward)
            optimizer.zero_grad()

            noise_pred = dit_model(z_noisy, t, text_emb, text_mask)

            #Loss Calculation (MSE)
            loss = F.mse_loss(noise_pred, noise)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            if step % 10 == 0:
                print(f"Epoch {epoch+1} | Step {step} | Loss: {loss.item():.4f}")
        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1} Complete. Avg Loss: {avg_loss:.4f}")
        torch.save(dit_model.state_dict(), f'ldmol_stage2_dit_epoch_{epoch+1}.pth')
    return dit_model

In [23]:
if __name__ == "__main__":
    enc_config = {
      "architectures": ["BertForMaskedLM"], "attention_probs_dropout_prob": 0.1,
      "hidden_act": "gelu", "hidden_dropout_prob": 0.1, 
      "hidden_size": 768, 
      "initializer_range": 0.02, "intermediate_size": 3072, "layer_norm_eps": 1e-12,
      "max_position_embeddings": 512, "model_type": "bert", "num_attention_heads": 12,
      "num_hidden_layers": 6, 
      "pad_token_id": 0, "type_vocab_size": 2, 
      "vocab_size": 300, 
      "fusion_layer": 6, "autoregressive": 0 
    }
    
    dec_config = {
      "architectures": ["BertForMaskedLM"], "attention_probs_dropout_prob": 0.1,
      "hidden_act": "gelu", "hidden_dropout_prob": 0.1, 
      "hidden_size": 512, 
      "initializer_range": 0.02, "intermediate_size": 2048, "layer_norm_eps": 1e-12,
      "max_position_embeddings": 512, "model_type": "bert", "num_attention_heads": 8, 
      "num_hidden_layers": 6, 
      "pad_token_id": 0, "type_vocab_size": 2, 
      "vocab_size": 300, 
      "fusion_layer": 0, 
      "encoder_width": 256, 
      "autoregressive": 0,
      "add_cross_attention": "True"
    }
    
    # Config for Stage 2 (You may need to lower batch_size for DiT + MolT5)
    train_cfg_stage2 = {
        'embed_dim': 512, 
        'batch_size': 8,  # Lowered to avoid OOM with MolT5-Large
        'schedular': {'epochs': 5}, 
    }
    # 2. Initialize Model & Load Weights
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Initializing model on {device}...")
    tokenizer = SMILESTokenizer(vocab_list)
    dummy_pt_config = {'embed_dim': 512, 'queue_size': 16384, 'momentum': 0.995, 'temp': 0.07}
    trained_model = LDMolAutoencoder(enc_config, dec_config, dummy_pt_config, tokenizer)
    checkpoint_path = '/kaggle/working/ldmol_stage1_epoch_5.pth'
    print(f"Loading weights from {checkpoint_path}...")
    
    checkpoint = torch.load(checkpoint_path, map_location='cpu') # Load to CPU first
    trained_model.load_state_dict(checkpoint['model_state_dict'])
    trained_model.eval()
    print("Stage 1 Model loaded successfully.")
    # Start Stage 2 Training
    import gc
    gc.collect()
    torch.cuda.empty_cache()
    
    print("Starting Stage 2: DiT Training...")
    dit_model = train_diffusion_stage(trained_model, train_cfg_stage2, ldmol_dataset, tokenizer)

Initializing model on cuda...
Loading weights from /kaggle/working/ldmol_stage1_epoch_5.pth...
Stage 1 Model loaded successfully.
Starting Stage 2: DiT Training...
Loading Text Encoder: laituan245/molt5-base...
Starting Diffusion (Stage 2) Training on cuda...
Epoch 1 | Step 0 | Loss: 2.2188
Epoch 1 | Step 10 | Loss: 1.4187
Epoch 1 | Step 20 | Loss: 0.9965
Epoch 1 | Step 30 | Loss: 0.9947
Epoch 1 | Step 40 | Loss: 0.9951
Epoch 1 | Step 50 | Loss: 0.9888
Epoch 1 | Step 60 | Loss: 0.9875
Epoch 1 | Step 70 | Loss: 0.9827
Epoch 1 | Step 80 | Loss: 0.9606
Epoch 1 | Step 90 | Loss: 0.8431
Epoch 1 | Step 100 | Loss: 0.7647
Epoch 1 | Step 110 | Loss: 0.7180
Epoch 1 | Step 120 | Loss: 0.6373
Epoch 1 | Step 130 | Loss: 0.5473
Epoch 1 | Step 140 | Loss: 0.4923
Epoch 1 | Step 150 | Loss: 0.5054
Epoch 1 | Step 160 | Loss: 0.4333
Epoch 1 | Step 170 | Loss: 0.4128
Epoch 1 | Step 180 | Loss: 0.4636
Epoch 1 | Step 190 | Loss: 0.3497
Epoch 1 | Step 200 | Loss: 0.3740
Epoch 1 | Step 210 | Loss: 0.3767
Epo

In [24]:
!pip install IPython

In [104]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import Draw

In [141]:
class LDMolInference:
    def __init__(self, stage1_model, dit_model, text_encoder, tokenizer, device='cuda'):
        self.stage1_model = stage1_model
        self.dit_model = dit_model
        self.text_encoder = text_encoder
        self.tokenizer = tokenizer
        self.device = device
        
        # Diffusion Constants
        self.num_timesteps = 1000
        self.betas = torch.linspace(0.0001, 0.02, self.num_timesteps).to(device)
        self.alphas = 1.0 - self.betas
        self.alphas_cumprod = torch.cumprod(self.alphas, dim=0)
        self.sqrt_inv_alphas = torch.sqrt(1.0 / self.alphas)
        
        # Set models to eval
        self.stage1_model.eval()
        self.dit_model.eval()

    @torch.no_grad()
    def sample_latent(self, text_prompt, num_samples=1, shape=(512, 512)):
        """
        Generates latent z from random noise conditioned on text.
        """
        # 1. Encode Text
        text_list = [text_prompt] * num_samples
        text_emb, text_mask = self.text_encoder.get_text_embeddings(text_list)
        
        # 2. Start from pure Gaussian noise
        batch_size = num_samples
        seq_len, dim = shape
        x = torch.randn(batch_size, seq_len, dim).to(self.device)
        
        # 3. Denoising Loop (Reverse Diffusion)
        print(f"Generating latent for: '{text_prompt}'...")
        for i in reversed(range(self.num_timesteps)):
            t = torch.full((batch_size,), i, device=self.device, dtype=torch.long)
            
            # Predict noise
            noise_pred = self.dit_model(x, t, text_emb, text_mask)
            
            # Compute previous image: x_{t-1} using DDPM formula
            alpha = self.alphas[i]
            alpha_cum = self.alphas_cumprod[i]
            beta = self.betas[i]
            
            mean = self.sqrt_inv_alphas[i] * (x - (beta / torch.sqrt(1 - alpha_cum)) * noise_pred)
            
            if i > 0:
                noise = torch.randn_like(x)
                sigma = torch.sqrt(beta)
                x = mean + sigma * noise
            else:
                x = mean
                
        return x

    @torch.no_grad()
    def decode_to_smiles(self, latent_z, max_length=128, temperature=0.7):
        """
        Decodes latent z into a SMILES string using Temperature Sampling.
        args:
            temperature: Controls randomness. 
                         < 1.0 makes it more conservative (like greedy).
                         > 1.0 makes it more random/diverse.
        """
        batch_size = latent_z.shape[0]
        
        # Start token is [CLS]
        start_token_id = self.tokenizer.cls_token_id
        input_ids = torch.tensor([[start_token_id]] * batch_size).to(self.device)
        
        generated_token_ids = []
        
        for _ in range(max_length):
            # Create attention mask
            attention_mask = torch.ones(input_ids.shape, device=self.device)
            
            # Forward pass through decoder
            outputs = self.stage1_model.decoder(
                input_ids=input_ids,
                attention_mask=attention_mask,
                encoder_hidden_states=latent_z,
                encoder_attention_mask=torch.ones(latent_z.shape[:2], device=self.device) 
            )
            
            # Get logits for the last token
            next_token_logits = outputs.logits[:, -1, :]
            
            # --- Sampling Logic ---
            # Apply temperature
            scaled_logits = next_token_logits / temperature
            # Convert to probabilities
            probs = torch.nn.functional.softmax(scaled_logits, dim=-1)
            
            # Sample from the distribution (instead of argmax)
            next_token_id = torch.multinomial(probs, num_samples=1)
            token_id = next_token_id.item()
            
            # Stop if [SEP]
            if token_id == self.tokenizer.sep_token_id:
                break
            
            # Safety break for start token loops
            if token_id == start_token_id:
                 continue # Just skip appending if it tries to restart, or break if you prefer

            generated_token_ids.append(token_id)
            input_ids = torch.cat([input_ids, next_token_id], dim=1)
            
        # --- Manual Decoding ---
        tokens = []
        for tid in generated_token_ids:
            token = self.tokenizer.id_to_token.get(tid, self.tokenizer.unk_token)
            tokens.append(token)
            
        smiles = "".join(tokens).replace("##", "")
        
        # Final cleanup
        smiles = smiles.replace(self.tokenizer.cls_token, "") \
                       .replace(self.tokenizer.sep_token, "") \
                       .replace(self.tokenizer.pad_token, "")
        
        return smiles

   

    def generate_molecule(self, text_prompt):
        latent = self.sample_latent(text_prompt)
        smiles = self.decode_to_smiles(latent)
        return smiles

In [142]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Loading Text Encoder for Inference...")
inference_text_encoder = MolT5TextEncoder('laituan245/molt5-base', device=device)

Loading Text Encoder for Inference...
Loading Text Encoder: laituan245/molt5-base...


In [144]:
sampler = LDMolInference(
    stage1_model=trained_model, 
    dit_model=dit_model,          
    text_encoder=inference_text_encoder,
    tokenizer=tokenizer,
    device=device
)

In [147]:
text_prompt = "Propadiene, stabilized appears as a colorless flammable gas. May become unstable. Not toxic but asphyxiation may occur because of displacement of oxygen. Under prolonged exposure to fire or heat the containers may rupture violently and rocket.\nThe molecule is a member of allenes."

In [148]:
print("\n--- Starting Generation ---")
generated_smiles = sampler.generate_molecule(text_prompt)

print(f"\nPrompt: {text_prompt}")
print(f"Generated SMILES: {generated_smiles}")


--- Starting Generation ---
Generating latent for: 'Propadiene, stabilized appears as a colorless flammable gas. May become unstable. Not toxic but asphyxiation may occur because of displacement of oxygen. Under prolonged exposure to fire or heat the containers may rupture violently and rocket.
The molecule is a member of allenes.'...

Prompt: Propadiene, stabilized appears as a colorless flammable gas. May become unstable. Not toxic but asphyxiation may occur because of displacement of oxygen. Under prolonged exposure to fire or heat the containers may rupture violently and rocket.
The molecule is a member of allenes.
Generated SMILES: 2+][UNK](C(=O)Nn2[UNK]c2c16COcc1[UNK]CC3)Cn1OC(C)]C22)c1c(N[UNK]CC1Zc4cccccc2)N1ZOC(C)c1)c2cccccO)c1ccccc1)cc1lc1cccc(CC


In [149]:
mol = Chem.MolFromSmiles(generated_smiles)
if mol:
    print("Valid Molecule Generated!")
    img = Draw.MolToImage(mol, size=(400, 400), legend=generated_smiles)
    plt.imshow(img)
    plt.axis('off')
    plt.show()

[16:29:39] SMILES Parse Error: syntax error while parsing: 2+][UNK](C(=O)Nn2[UNK]c2c16COcc1[UNK]CC3)Cn1OC(C)]C22)c1c(N[UNK]CC1Zc4cccccc2)N1ZOC(C)c1)c2cccccO)c1ccccc1)cc1lc1cccc(CC
[16:29:39] SMILES Parse Error: check for mistakes around position 1:
[16:29:39] 2+][UNK](C(=O)Nn2[UNK]c2c16COcc1[UNK]CC3)
[16:29:39] ^
[16:29:39] SMILES Parse Error: Failed parsing SMILES '2+][UNK](C(=O)Nn2[UNK]c2c16COcc1[UNK]CC3)Cn1OC(C)]C22)c1c(N[UNK]CC1Zc4cccccc2)N1ZOC(C)c1)c2cccccO)c1ccccc1)cc1lc1cccc(CC' for input: '2+][UNK](C(=O)Nn2[UNK]c2c16COcc1[UNK]CC3)Cn1OC(C)]C22)c1c(N[UNK]CC1Zc4cccccc2)N1ZOC(C)c1)c2cccccO)c1ccccc1)cc1lc1cccc(CC'
